## Load packages:<a href="#Load-packages:" class="anchor-link">¶</a>

In \[1\]:

    library(dplyr)
    library(ggplot2)
    library(forecast)

    Attaching package: ‘dplyr’


    The following objects are masked from ‘package:stats’:

        filter, lag


    The following objects are masked from ‘package:base’:

        intersect, setdiff, setequal, union


    Registered S3 method overwritten by 'quantmod':
      method            from
      as.zoo.data.frame zoo 

## Part 1: Data<a href="#Part-1:-Data" class="anchor-link">¶</a>

I upload the time series from 1900 to 2021 of the disasters in the
world:

**Load data:**

In \[2\]:

    data <- read.csv("../input/all-natural-disasters-19002021-eosdis/DISASTERS/1900_2021_DISASTERS.xlsx - emdat data.csv", stringsAsFactors = T)

There are missing values in the dataset but Year, Disaster.Type,
Disaster.Subtype and Continent variables don't have any, so I don't
consider the missing values problem:

In \[3\]:

    colSums(is.na(data))

Year  
0

Seq  
0

Glide  
0

Disaster.Group  
0

Disaster.Subgroup  
0

Disaster.Type  
0

Disaster.Subtype  
0

Disaster.Subsubtype  
0

Event.Name  
0

Country  
0

ISO  
0

Region  
0

Continent  
0

Location  
0

Origin  
0

Associated.Dis  
0

Associated.Dis2  
0

OFDA.Response  
0

Appeal  
0

Declaration  
0

Aid.Contribution  
15449

Dis.Mag.Value  
11180

Dis.Mag.Scale  
0

Latitude  
0

Longitude  
0

Local.Time  
0

River.Basin  
0

Start.Year  
0

Start.Month  
387

Start.Day  
3628

End.Year  
0

End.Month  
708

End.Day  
3556

Total.Deaths  
4713

No.Injured  
12231

No.Affected  
6906

No.Homeless  
13696

Total.Affected  
4509

Insured.Damages...000.US..  
15030

Total.Damages...000.US..  
10881

CPI  
315

Adm.Level  
0

Admin1.Code  
0

Admin2.Code  
0

Geo.Locations  
0

Values of categorical variables in the dataset:

In \[4\]:

    levels(data$Disaster.Subgroup)
    print("Disaster.Type:")
    levels(data$Disaster.Type)
    print("Disaster.Subtype:")
    levels(data$Disaster.Subtype)
    print("Disaster.Subsubtype:")
    levels(data$Disaster.Subsubtype)
    print("Continent:")
    levels(data$Continent)
    print("Region:")
    levels(data$Region)

1.  'Biological'
2.  'Climatological'
3.  'Extra-terrestrial'
4.  'Geophysical'
5.  'Hydrological'
6.  'Meteorological'

    [1] "Disaster.Type:"

1.  'Animal accident'
2.  'Drought'
3.  'Earthquake'
4.  'Epidemic'
5.  'Extreme temperature '
6.  'Flood'
7.  'Fog'
8.  'Glacial lake outburst'
9.  'Impact'
10. 'Insect infestation'
11. 'Landslide'
12. 'Mass movement (dry)'
13. 'Storm'
14. 'Volcanic activity'
15. 'Wildfire'

    [1] "Disaster.Subtype:"

1.  ''
2.  'Ash fall'
3.  'Avalanche'
4.  'Bacterial disease'
5.  'Coastal flood'
6.  'Cold wave'
7.  'Convective storm'
8.  'Drought'
9.  'Extra-tropical storm'
10. 'Flash flood'
11. 'Forest fire'
12. 'Grasshopper'
13. 'Ground movement'
14. 'Heat wave'
15. 'Land fire (Brush, Bush, Pasture)'
16. 'Landslide'
17. 'Lava flow'
18. 'Locust'
19. 'Mudslide'
20. 'Parasitic disease'
21. 'Pyroclastic flow '
22. 'Riverine flood'
23. 'Rockfall'
24. 'Severe winter conditions'
25. 'Subsidence'
26. 'Tropical cyclone'
27. 'Tsunami'
28. 'Viral disease'

    [1] "Disaster.Subsubtype:"

1.  ''
2.  'Derecho'
3.  'Hail'
4.  'Lightning/Thunderstorms'
5.  'Mudslide'
6.  'Rain'
7.  'Sand/Dust storm'
8.  'Severe storm'
9.  'Snow/Ice'
10. 'Storm/Surge'
11. 'Sudden subsidence'
12. 'Tornado'
13. 'Winter storm/Blizzard'

    [1] "Continent:"

1.  'Africa'
2.  'Americas'
3.  'Asia'
4.  'Europe'
5.  'Oceania'

    [1] "Region:"

1.  'Australia and New Zealand'
2.  'Caribbean'
3.  'Central America'
4.  'Central Asia'
5.  'Eastern Africa'
6.  'Eastern Asia'
7.  'Eastern Europe'
8.  'Melanesia'
9.  'Micronesia'
10. 'Middle Africa'
11. 'Northern Africa'
12. 'Northern America'
13. 'Northern Europe'
14. 'Polynesia'
15. 'Russian Federation'
16. 'South America'
17. 'South-Eastern Asia'
18. 'Southern Africa'
19. 'Southern Asia'
20. 'Southern Europe'
21. 'Western Africa'
22. 'Western Asia'
23. 'Western Europe'

## Part 2: Research question<a href="#Part-2:-Research-question" class="anchor-link">¶</a>

We want to predict the frequency of the main disasters (Floods and
Storms) from 2022 onwards and evaluate if there is an increasing trend
in the time series of these disasters.

## Part 3: EDA<a href="#Part-3:-EDA" class="anchor-link">¶</a>

As can be seen from the following graph, the most frequent disasters in
the dataset are Floods and Storms

In \[5\]:

    df<-data %>%
      group_by(Disaster.Type) %>%
      summarise(total=n()) 

    df %>%
    mutate(Disaster.Type=reorder(Disaster.Type,total))  %>%
      ggplot(aes(Disaster.Type,total, fill=Disaster.Type)) +
      geom_bar(stat = "identity")+
      coord_flip()+
      geom_text(aes(label=total), hjust=0, size=2)+
      guides(fill="none")

<img src="...%0AJAwyZ40AAAAASUVORK5CYII=" width="420" height="420" />

An increasing trend of these disasters in the time series from 1900 to
2021 and also of tropical cyclones can be seen from the following
graphs:

In \[6\]:

    data %>%
      filter(Disaster.Type==c("Flood","Storm")) %>%
      group_by(Disaster.Type, Year) %>%
      summarise(total=n()) %>%
      ggplot(aes(Year,total, colour=Disaster.Type)) +
      geom_line()

    `summarise()` has grouped output by 'Disaster.Type'. You can override using the `.groups` argument.

<img src="..." width="420" height="420" />

In \[7\]:

    data %>%
      filter(Disaster.Subtype=="Tropical cyclone") %>%
      group_by(Year) %>%
      summarise(total=n()) %>%
      ggplot(aes(Year,total)) +
      geom_line(colour="Orange")+
      ggtitle("Time series tropical cyclone")

<img src="...%0AAABJRU5ErkJggg==" width="420" height="420" />

I create a function to evaluate the total number of Floods and Storms
per continent and the related time series and it is noted that the
trends are growing for all continents and then stabilize in recent
years:

In \[8\]:

    View_disaster_per_continent <- function(Disaster) {
      
      df<-data %>%
        filter(Disaster.Type==Disaster) %>%
        group_by(Continent) %>%
        summarise(total=n()) %>%
        arrange(desc(total))
      
      
      g<- data %>%
        filter(Disaster.Type==Disaster) %>%
        group_by(Continent,Year) %>%
        summarise(total=n()) %>%
        ggplot(aes(Year,total, colour=Continent)) +
        geom_line()+
        ggtitle(Disaster)
      
      print(df)
      print(g)
    }

In \[9\]:

    View_disaster_per_continent("Flood")

    `summarise()` has grouped output by 'Continent'. You can override using the `.groups` argument.

    # A tibble: 5 × 2
      Continent total
      <fct>     <int>
    1 Asia       2303
    2 Americas   1275
    3 Africa     1147
    4 Europe      668
    5 Oceania     158

<img src="..." width="420" height="420" />

In \[10\]:

    View_disaster_per_continent("Storm")

    `summarise()` has grouped output by 'Continent'. You can override using the `.groups` argument.

    # A tibble: 5 × 2
      Continent total
      <fct>     <int>
    1 Asia       1884
    2 Americas   1442
    3 Europe      542
    4 Oceania     338
    5 Africa      290

<img src="..." width="420" height="420" />

In \[11\]:

    df<-data %>%
      filter(Disaster.Type=="Storm") %>%
      group_by(Country) %>%
      summarise(total=n()) 

    g<-df %>%
    filter(total>15) %>%  
    mutate(Country=reorder(Country,total))  %>%
      ggplot(aes(Country,total, fill=Country)) +
      geom_bar(stat = "identity")+
      coord_flip()+
      geom_text(aes(label=total), hjust=0, size=2)+
      guides(fill="none")+
      ggtitle("Total number of Storms per country", subtitle="from 1901 to 2021")

    df<-data %>%
      filter(Disaster.Type=="Flood") %>%
      group_by(Country) %>%
      summarise(total=n()) 

    print(g)

    g<-df %>%
    filter(total>30) %>%  
    mutate(Country=reorder(Country,total))  %>%
      ggplot(aes(Country,total, fill=Country)) +
      geom_bar(stat = "identity")+
      coord_flip()+
      geom_text(aes(label=total), hjust=0, size=2)+
      guides(fill="none")+
      ggtitle("Total number of Floods per country", subtitle="from 1901 to 2021")

    print(g)

<img src="...%0AYII=" width="420" height="420" />

<img src="..." width="420" height="420" />

## Part 4: Forecasting<a href="#Part-4:-Forecasting" class="anchor-link">¶</a>

I compare the ets forecasting model with the arima model by calculating
the forecast errors by dividing the time series into a train and a test.
It is noted that the arima model gives minor errors on the test and
therefore is better:

In \[12\]:

    dis<-data %>%
        filter(Disaster.Type=="Storm") %>%
        group_by(Year) %>%
        summarise(total=n()) 

    #training and test sets
    train <- ts(dis[1:105,2])
    test <- ts(dis[106:118,2])

In \[13\]:

    fc_ets <- forecast(train,12)
    acc_ets <- accuracy(fc_ets, test[1:12])
    acc_ets

|              | ME        | RMSE     | MAE       | MPE       | MAPE     | MASE      | ACF1       |
|--------------|-----------|----------|-----------|-----------|----------|-----------|------------|
| Training set | 1.152539  | 11.25402 | 5.671677  | -23.07858 | 41.24319 | 0.8573465 | -0.2161825 |
| Test set     | -9.097916 | 17.43204 | 16.439080 | -11.28687 | 17.06459 | 2.4849771 | NA         |

A matrix: 2 × 7 of type dbl

In \[14\]:

    ar <- auto.arima(train)
    fc_arima <- forecast(ar,12)
    acc_arima <- accuracy(fc_arima, test[1:12])
    acc_arima

|              | ME           | RMSE     | MAE       | MPE        | MAPE     | MASE      | ACF1        |
|--------------|--------------|----------|-----------|------------|----------|-----------|-------------|
| Training set | -0.006098709 | 10.11330 | 5.286694  | -46.551056 | 56.65213 | 0.7991514 | 0.005505753 |
| Test set     | -5.865769247 | 15.49559 | 13.954389 | -7.832744  | 14.22381 | 2.1093844 | NA          |

A matrix: 2 × 7 of type dbl

I create a function that predicts the total number of Floods and Storms
in the world from 2022 to 2032 with an 80% and 95% confidence interval
with arima model:

In \[15\]:

    forecast_disaster <- function( Disaster) {
      dis<-data %>%
        filter(Disaster.Type==Disaster) %>%
        group_by(Year) %>%
        summarise(total=n()) 
      
      serie <-ts(dis$total)
      ar <- auto.arima(serie)
      fore<-forecast(ar)
      
      plot(fore)
      
      year<-2022:2031
      df_pred <-data.frame(q=99:108)
      p<-predict (fore,newdata=df_pred, se=TRUE)
      cat("Forecasting",Disaster,"\n")
      cbind(as.data.frame(year),p)
     }

In \[16\]:

    forecast_disaster("Flood")

    Forecasting Flood 

|     | year   | Point Forecast | Lo 80    | Hi 80    | Lo 95    | Hi 95    |
|-----|--------|----------------|----------|----------|----------|----------|
|     | \<int> | \<dbl>         | \<dbl>   | \<dbl>   | \<dbl>   | \<dbl>   |
| 99  | 2022   | 163.3080       | 144.2550 | 182.3611 | 134.1689 | 192.4472 |
| 100 | 2023   | 186.3730       | 160.3882 | 212.3578 | 146.6326 | 226.1133 |
| 101 | 2024   | 185.8720       | 158.6080 | 213.1359 | 144.1754 | 227.5685 |
| 102 | 2025   | 177.2149       | 148.3008 | 206.1291 | 132.9946 | 221.4353 |
| 103 | 2026   | 180.9210       | 148.6942 | 213.1477 | 131.6345 | 230.2075 |
| 104 | 2027   | 187.8025       | 152.9734 | 222.6317 | 134.5360 | 241.0691 |
| 105 | 2028   | 188.2784       | 151.7580 | 224.7989 | 132.4253 | 244.1316 |
| 106 | 2029   | 187.6336       | 149.2979 | 225.9693 | 129.0042 | 246.2630 |
| 107 | 2030   | 190.2696       | 149.8789 | 230.6603 | 128.4973 | 252.0419 |
| 108 | 2031   | 193.2271       | 151.0031 | 235.4511 | 128.6511 | 257.8031 |

A data.frame: 10 × 6

<img src="..." width="420" height="420" />

In \[17\]:

    forecast_disaster("Storm")

    Forecasting Storm 

|     | year   | Point Forecast | Lo 80    | Hi 80    | Lo 95    | Hi 95    |
|-----|--------|----------------|----------|----------|----------|----------|
|     | \<int> | \<dbl>         | \<dbl>   | \<dbl>   | \<dbl>   | \<dbl>   |
| 119 | 2022   | 106.96683      | 92.58186 | 121.3518 | 84.96692 | 128.9667 |
| 120 | 2023   | 112.42125      | 97.74714 | 127.0954 | 89.97913 | 134.8634 |
| 121 | 2024   | 95.83292       | 80.75905 | 110.9068 | 72.77942 | 118.8864 |
| 122 | 2025   | 112.64411      | 95.00880 | 130.2794 | 85.67323 | 139.6150 |
| 123 | 2026   | 106.25835      | 88.39740 | 124.1193 | 78.94238 | 133.5743 |
| 124 | 2027   | 106.08475      | 87.11730 | 125.0522 | 77.07653 | 135.0930 |
| 125 | 2028   | 112.04641      | 92.12455 | 131.9683 | 81.57856 | 142.5143 |
| 126 | 2029   | 108.13206      | 87.71808 | 128.5460 | 76.91157 | 139.3526 |
| 127 | 2030   | 111.28626      | 89.90959 | 132.6629 | 78.59346 | 143.9791 |
| 128 | 2031   | 112.38255      | 90.37158 | 134.3935 | 78.71967 | 146.0454 |

A data.frame: 10 × 6

<img src="...%0Agg==" width="420" height="420" />

In \[ \]:

     

In \[ \]:

     

In \[ \]: